In [1]:
import pandas as pd
import isodate
import re

In [2]:
df = pd.read_csv("./unprocessed_recipes_data.csv")

In [3]:
df.head()

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan..."
1,39,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29T13:12:00Z,Make and share this Biryani recipe from Food.com.,"c(""https://img.sndimg.com/food/image/upload/w_...",...,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,NaN,"c(""Soak saffron in warm milk for 5 minutes and..."
2,40,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05T19:52:00Z,This is from one of my first Good House Keepi...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,NaN,"c(""Into a 1 quart Jar with tight fitting lid, ..."
3,41,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03T14:54:00Z,This dish is best prepared a day in advance to...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,4 kebabs,"c(""Drain the tofu, carefully squeezing out exc..."
4,42,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19T06:19:00Z,Make and share this Cabbage Soup recipe from F...,"""https://img.sndimg.com/food/image/upload/w_55...",...,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,NaN,"c(""Mix everything together and bring to a boil..."


In [4]:
df = df.drop(['RecipeId', 'AuthorId', 'AuthorName', 'DatePublished', 'Description',
             'Images', 'RecipeCategory', 'Keywords', 'AggregatedRating', 'ReviewCount',
             'SaturatedFatContent', 'CholesterolContent', 'CarbohydrateContent', 'FiberContent',
             'RecipeYield'], axis=1)

In [5]:
print(df.columns)
print(df.shape)

Index(['Name', 'CookTime', 'PrepTime', 'TotalTime',
       'RecipeIngredientQuantities', 'RecipeIngredientParts', 'Calories',
       'FatContent', 'SodiumContent', 'SugarContent', 'ProteinContent',
       'RecipeServings', 'RecipeInstructions'],
      dtype='object')
(522517, 13)


In [6]:
df = df.dropna()
print(df.shape)

(287338, 13)


In [7]:
df.tail()

,Name,CookTime,PrepTime,TotalTime,RecipeIngredientQuantities,RecipeIngredientParts,Calories,FatContent,SodiumContent,SugarContent,ProteinContent,RecipeServings,RecipeInstructions
522508,Amazing Ground Beef Stroganoff,PT20M,PT30M,PT50M,"c(""1"", ""1"", ""1"", ""1/2"", ""1/4"", ""1"", ""1"", ""1"", ...","c(""hamburger"", ""onion"", ""celery"", ""water chest...",422.3,28.6,634.7,5.7,27.3,4.0,"c(""Saute meat in a medium skillet until it los..."
522510,Slow-Cooker Classic Coffee Cake,PT3H,PT20M,PT3H20M,"c(""1"", ""1/2"", ""4"", ""2"", ""1/8"", ""1"", ""1"", ""1/2""...","c(""all-purpose flour"", ""brown sugar"", ""butter""...",358.9,19.8,323.4,24.8,4.8,12.0,"c(""Line bottom and sides of 5-quart oval slow ..."
522512,Meg's Fresh Ginger Gingerbread,PT35M,PT1H,PT1H35M,"c(""3"", ""1/2"", ""1/2"", ""1/4"", ""1/4"", ""1"", ""1/4"",...","c(""fresh ginger"", ""unsalted butter"", ""dark bro...",316.6,12.5,278.2,22.8,3.9,8.0,"c(""Preheat oven to 350&deg;F Grease an 8x8 cak..."
522513,Roast Prime Rib au Poivre with Mixed Peppercorns,PT3H,PT30M,PT3H30M,"c(""9"", ""2"", ""4"", ""2"", ""1/3"", ""3 1/2"", ""1/3"")","c(""Dijon mustard"", ""garlic"", ""peppercorns"", ""s...",2063.4,172.4,766.3,0.1,117.0,8.0,"c(""Position rack in center of oven and preheat..."
522514,Kirshwasser Ice Cream,PT3H,PT1H,PT4H,"c(""3"", ""3"", ""1/2"", ""1/2"", ""2"", ""2"")","c(""half-and-half"", ""heavy cream"", ""brandy"", ""s...",1271.3,117.2,192.5,17.3,12.8,6.0,"c(""heat half and half and heavy cream to a sim..."


In [8]:
# Function to convert ISO 8601 durations to a human-readable format with singular/plural handling
def convert_duration_to_minutes(duration_string):
    if '-' in duration_string:
        # Remove the negative sign and parse the duration
        duration_string = duration_string.replace('-', '')
    # Parse the ISO 8601 duration string
    duration = isodate.parse_duration(duration_string)
    
    # Get the total seconds from the duration
    total_seconds = duration.total_seconds()

    return_minutes = total_seconds / 60
    
    # Calculate the hours and minutes separately
    total_minutes = total_seconds // 60
    hours = total_minutes // 60
    minutes = total_minutes % 60

    # Build the output string conditionally based on the hours and minutes
    result = []
    
    if hours > 0:
        hour_str = "hour" if hours == 1 else "hours"
        result.append(f"{int(hours)} {hour_str}")
    
    if minutes > 0:
        minute_str = "minute" if minutes == 1 else "minutes"
        result.append(f"{int(minutes)} {minute_str}")

    # Join the results (if both hours and minutes are present) and return the output string
    return ' '.join(result), return_minutes

In [9]:
df['CookTime'], df['CookTimeMinutes'] = zip(*df['CookTime'].apply(convert_duration_to_minutes))

In [10]:
df['PrepTime'], df['PrepTimeMinutes'] = zip(*df['PrepTime'].apply(convert_duration_to_minutes))

In [11]:
df['TotalTime'], df['TotalTimeMinutes'] = zip(*df['TotalTime'].apply(convert_duration_to_minutes))

In [12]:
# Function to convert the string into a list by cleaning it up
def string_to_list(s):
    # Remove 'c(' and ')'
    s = s.replace('c(', '').replace(')', '')
    
    # Use regex to split on commas only outside of quotes
    items = re.split(r',\s*(?=(?:[^"]*"[^"]*")*[^"]*$)', s)
    
    # Strip the quotes from each item and return the clean list
    return [item.strip().replace('"', '') for item in items]

# Apply the conversion to both columns
df['RecipeIngredientQuantities'] = df['RecipeIngredientQuantities'].apply(string_to_list)
df['RecipeIngredientParts'] = df['RecipeIngredientParts'].apply(string_to_list)

# Combine quantities and ingredients into a single string
def combine_ingredients(row):
    combined = [f"{qty} {ingr}" for qty, ingr in zip(row['RecipeIngredientQuantities'], row['RecipeIngredientParts'])]
    return ', '.join(combined)

# Apply the combination function to each row
df['Ingredients'] = df.apply(combine_ingredients, axis=1)

In [13]:
df.head()

,Name,CookTime,PrepTime,TotalTime,RecipeIngredientQuantities,RecipeIngredientParts,Calories,FatContent,SodiumContent,SugarContent,ProteinContent,RecipeServings,RecipeInstructions,CookTimeMinutes,PrepTimeMinutes,TotalTimeMinutes,Ingredients
0,Low-Fat Berry Blue Frozen Dessert,24 hours,45 minutes,24 hours 45 minutes,"[4, 1/4, 1, 1]","[blueberries, granulated sugar, vanilla yogurt...",170.9,2.5,29.8,30.2,3.2,4.0,"c(""Toss 2 cups berries with sugar."", ""Let stan...",1440.0,45.0,1485.0,"4 blueberries, 1/4 granulated sugar, 1 vanilla..."
1,Biryani,25 minutes,4 hours,4 hours 25 minutes,"[1, 4, 2, 2, 8, 1/4, 8, 1/2, 1, 1, 1/4, 1/4, 1...","[saffron, milk, hot green chili peppers, onion...",1110.7,58.8,368.4,20.4,63.4,6.0,"c(""Soak saffron in warm milk for 5 minutes and...",25.0,240.0,265.0,"1 saffron, 4 milk, 2 hot green chili peppers, ..."
2,Best Lemonade,5 minutes,30 minutes,35 minutes,"[1 1/2, 1, NA, 1 1/2, NA, 3/4]","[sugar, lemons, rind of, lemon, zest of, fresh...",311.1,0.2,1.8,77.2,0.3,4.0,"c(""Into a 1 quart Jar with tight fitting lid, ...",5.0,30.0,35.0,"1 1/2 sugar, 1 lemons, rind of, NA lemon, zest..."
3,Carina's Tofu-Vegetable Kebabs,20 minutes,24 hours,24 hours 20 minutes,"[12, 1, 2, 1, 10, 1, 3, 2, 2, 2, 1, 2, 1/2, 1/...","[extra firm tofu, eggplant, zucchini, mushroom...",536.1,24.0,1558.6,32.1,29.3,2.0,"c(""Drain the tofu, carefully squeezing out exc...",20.0,1440.0,1460.0,"12 extra firm tofu, 1 eggplant, 2 zucchini, 1 ..."
4,Cabbage Soup,30 minutes,20 minutes,50 minutes,"[46, 4, 1, 2, 1]","[plain tomato juice, cabbage, onion, carrots, ...",103.6,0.4,959.3,17.7,4.3,4.0,"c(""Mix everything together and bring to a boil...",30.0,20.0,50.0,"46 plain tomato juice, 4 cabbage, 1 onion, 2 c..."


In [14]:
# Function to extract and format the steps
def format_instructions(instructions):
    # Step 1: Remove c( and ), clean newlines, and handle other characters before quotes
    cleaned_instructions = instructions.replace('c(', '').replace(')', '')
    
    # Step 2: Use regex to capture text inside the quotes only
    steps = re.findall(r'"(.*?)"', cleaned_instructions)
    
    # Step 3: Clean each step to remove unwanted characters like newlines, tabs, etc.
    cleaned_steps = [step.strip().replace('\n', '').replace('"', '') for step in steps]
    
    # Step 4: Number the steps
    formatted_steps = [f"{i+1}. {step}" for i, step in enumerate(cleaned_steps)]
    
    # Return formatted steps joined with line breaks
    return "\n".join(formatted_steps)

# Apply the function to the 'RecipeInstructions' column
df['FormattedRecipeInstructions'] = df['RecipeInstructions'].apply(format_instructions)

In [15]:
df.columns

Index(['Name', 'CookTime', 'PrepTime', 'TotalTime',
       'RecipeIngredientQuantities', 'RecipeIngredientParts', 'Calories',
       'FatContent', 'SodiumContent', 'SugarContent', 'ProteinContent',
       'RecipeServings', 'RecipeInstructions', 'CookTimeMinutes',
       'PrepTimeMinutes', 'TotalTimeMinutes', 'Ingredients',
       'FormattedRecipeInstructions'],
      dtype='object')

In [16]:
df = df.drop(['RecipeInstructions', 'RecipeIngredientQuantities'], axis=1)

In [17]:
# Save the DataFrame to a CSV file
df.to_csv('recipes_clean.csv', index=False)

In [27]:
df_test = df.head(1)

In [28]:
df_test

,Name,CookTime,PrepTime,TotalTime,RecipeIngredientParts,Calories,FatContent,SodiumContent,SugarContent,ProteinContent,RecipeServings,CookTimeMinutes,PrepTimeMinutes,TotalTimeMinutes,Ingredients,FormattedRecipeInstructions
0,Low-Fat Berry Blue Frozen Dessert,24 hours,45 minutes,24 hours 45 minutes,"[blueberries, granulated sugar, vanilla yogurt...",170.9,2.5,29.8,30.2,3.2,4.0,1440.0,45.0,1485.0,"4 blueberries, 1/4 granulated sugar, 1 vanilla...",1. Toss 2 cups berries with sugar.\n2. Let sta...


In [29]:
# Save the DataFrame to a CSV file
df_test.to_csv('recipes_test.csv', index=False)